In [3]:
import taichi as ti

from data_structures import SimulationParams, PhysicsParams
from print_ import Printing


ti.init(arch=ti.cpu)

sim = SimulationParams()
phys = PhysicsParams()

printer = Printing(sim, phys, output_dir="./result", stdout=True)

printer.open_files()
printer.start_time()

# Just to test basic lifecycle
printer.itertot = 10
printer.cal_time()
printer.end_time()

print("OK: printing module ran.")

[Taichi] Starting on arch=x64
TITLE = "Thermo-Capillary Flow in Laser-Generated Melt Pool"
  Date: 2026-01-30  time: 19 :53 :20
  Date: 2026-01-30  time: 19 :53 :20
  Total time used:     0  hr     0  m     0  s
OK: printing module ran.


In [4]:
# test_laser.py
import taichi as ti
import numpy as np

from data_structures import SimulationParams, PhysicsParams, GridParams, LaserParams, ToolPath, LaserState
from laser import LaserInput


def main():
    ti.init(arch=ti.cpu)

    # Use smaller grid for quick test
    sim = SimulationParams(ni=64, nj=64, nk=16, xlen=1.0e-3, ylen=1.0e-3, zlen=2.0e-4)
    phys = PhysicsParams()

    grid = GridParams(sim.ni, sim.nj, sim.nk)
    laser_params = LaserParams(power=200.0, radius=50.0e-6, absorptivity=0.35, efficiency=1.0)
    laser_state = LaserState(sim.ni, sim.nj)

    # Build a simple toolpath: move along x, constant y
    # Note: time must be increasing; laser_on must be 0/1 (or >=0.5 check)
    toolpath = ToolPath(
        time=np.array([0.0, 2.0e-5, 4.0e-5, 6.0e-5], dtype=np.float64),
        x=np.array([2.0e-4, 3.0e-4, 4.0e-4, 5.0e-4], dtype=np.float64),
        y=np.array([5.0e-4, 5.0e-4, 5.0e-4, 5.0e-4], dtype=np.float64),
        z=np.array([0.0, 0.0, 0.0, 0.0], dtype=np.float64),
        laser_on=np.array([1, 1, 1, 1], dtype=np.int32),
        n_segments=4,
    )

    # Initialize grid coordinates and areaij
    dx = sim.xlen / (sim.ni - 1)
    dy = sim.ylen / (sim.nj - 1)

    @ti.kernel
    def init_grid():
        for i in range(sim.ni):
            grid.x[i] = ti.cast(i, ti.f64) * dx
        for j in range(sim.nj):
            grid.y[j] = ti.cast(j, ti.f64) * dy
        for k in range(sim.nk):
            grid.z[k] = ti.cast(k, ti.f64) * (sim.zlen / max(sim.nk - 1, 1))

        for i, j in ti.ndrange(sim.ni, sim.nj):
            grid.areaij[i, j] = dx * dy

    init_grid()

    # Create laser module and init beam position from toolpath
    laser = LaserInput(sim, grid, laser_params, toolpath, laser_state, gaussian_factor=2.0)
    laser.init_from_toolpath(start_segment=0)

    # Run a few timesteps to see it move
    for step in range(5):
        timet = step * sim.delt
        heatin_laser, pool_box = laser.laser_beam(timet)

        heat_np = laser_state.heatin.to_numpy()
        max_q = float(np.max(heat_np))
        argmax = np.unravel_index(np.argmax(heat_np), heat_np.shape)

        print(f"\nStep {step} timet={timet:.3e}")
        print(f"  beam_x={laser_state.beam_x:.6e}, beam_y={laser_state.beam_y:.6e}, laser_on={laser_state.laser_on}")
        print(f"  heatinLaser(total)={heatin_laser:.6e}")
        print(f"  max(heatin)={max_q:.6e} at (i,j)={argmax}")
        print(f"  pool_box={pool_box}")

    # Optional: save last heat field for inspection
    np.save("heatin_last.npy", laser_state.heatin.to_numpy())
    print("\nSaved heatin_last.npy")


if __name__ == "__main__":
    main()


[Taichi] Starting on arch=x64

Step 0 timet=0.000e+00
  beam_x=2.000000e-04, beam_y=5.000000e-04, laser_on=True
  heatinLaser(total)=6.999999e+01
  max(heatin)=1.641149e+10 at (i,j)=(np.int64(13), np.int64(31))
  pool_box=PoolIndexBox(istart=13, jstart=31, imin=11, imax=15, jmin=29, jmax=33, kmin=11, kmax=14)

Step 1 timet=1.000e-06
  beam_x=2.050000e-04, beam_y=5.000000e-04, laser_on=True
  heatinLaser(total)=6.999999e+01
  max(heatin)=1.692472e+10 at (i,j)=(np.int64(13), np.int64(31))
  pool_box=PoolIndexBox(istart=13, jstart=31, imin=11, imax=15, jmin=29, jmax=33, kmin=11, kmax=14)

Step 2 timet=2.000e-06
  beam_x=2.100000e-04, beam_y=5.000000e-04, laser_on=True
  heatinLaser(total)=6.999999e+01
  max(heatin)=1.676962e+10 at (i,j)=(np.int64(13), np.int64(31))
  pool_box=PoolIndexBox(istart=13, jstart=31, imin=11, imax=15, jmin=29, jmax=33, kmin=11, kmax=14)

Step 3 timet=3.000e-06
  beam_x=2.150000e-04, beam_y=5.000000e-04, laser_on=True
  heatinLaser(total)=6.999999e+01
  max(heati

In [7]:
import taichi as ti
import numpy as np

from data_structures import SimulationParams, PhysicsParams, GridParams, State, MaterialProps
from prop import PropertyModel, PropertyCoeffs, make_solidfield_placeholder


def main():
    ti.init(arch=ti.cpu)

    # Small grid for quick test
    sim = SimulationParams(ni=8, nj=7, nk=6, xlen=1e-3, ylen=1e-3, zlen=2e-4)
    phys = PhysicsParams(
        # Keep defaults unless you want to match your Fortran constants
        acpa=0.0,
        acpb=500.0,
        acpl=800.0,
        tsolid=1563.0,
        tliquid=1623.0,
        rho=7800.0,
        rholiq=7800.0,
        tcond=25.0,
        tcondl=25.0,
        vis0=6e-3,
    )

    grid = GridParams(sim.ni, sim.nj, sim.nk)
    state = State(sim.ni, sim.nj, sim.nk)
    props = MaterialProps(sim.ni, sim.nj, sim.nk)

    # solidfield is required by the powder branch
    solidfield = make_solidfield_placeholder(sim)

    # Initialize z so (z_top - z[k]) check works
    dz = sim.zlen / max(sim.nk - 1, 1)

    @ti.kernel
    def init_z():
        for k in range(sim.nk):
            grid.z[k] = ti.cast(k, ti.f64) * dz

    init_z()

    # Build coefficients used by properties()
    coeffs = PropertyCoeffs.from_physics(phys, layerheight=dz * 1.5)  # top ~ 1-2 layers treated as powder region
    coeffs.dens = 7800.0
    coeffs.denl = 7800.0
    coeffs.viscos = 6e-3
    coeffs.pden = 4000.0
    coeffs.pthcona = 0.0
    coeffs.pthconb = 5.0
    coeffs.pcpa = 0.0
    coeffs.pcpb = 500.0
    coeffs.thconsa = 0.0
    coeffs.thconsb = 25.0
    coeffs.thconl = 25.0

    # Set up 3 temperature zones to hit all branches:
    #   - Liquid: T >= tliquid
    #   - Solid:  T <= tsolid
    #   - Mushy:  tsolid < T < tliquid with fracl
    @ti.kernel
    def init_state_and_solidfield():
        for i, j, k in ti.ndrange(sim.ni, sim.nj, sim.nk):
            # Default: solid temperature
            state.temp[i, j, k] = phys.tsolid - 50.0
            state.fracl[i, j, k] = 0.0
            solidfield[i, j, k] = 1.0  # treat as solid

        # Make a small liquid pocket at center
        for i, j, k in ti.ndrange(sim.ni, sim.nj, sim.nk):
            if i == sim.ni // 2 and j == sim.nj // 2:
                state.temp[i, j, k] = phys.tliquid + 50.0
                state.fracl[i, j, k] = 1.0
                solidfield[i, j, k] = 0.0

        # Make a mushy line at i=1 with fracl=0.5
        for j, k in ti.ndrange(sim.nj, sim.nk):
            state.temp[1, j, k] = 0.5 * (phys.tsolid + phys.tliquid)
            state.fracl[1, j, k] = 0.5
            solidfield[1, j, k] = 0.0

        # Mark top layer as powder-eligible: solidfield <= 0.5 near top
        for i, j in ti.ndrange(sim.ni, sim.nj):
            solidfield[i, j, sim.nk - 1] = 0.0
            # Keep it solid temperature so it goes into the powder branch
            state.temp[i, j, sim.nk - 1] = phys.tsolid - 100.0
            state.fracl[i, j, sim.nk - 1] = 0.0

    init_state_and_solidfield()

    model = PropertyModel(sim, phys, grid, state, props, solidfield, coeffs)
    model.update()

    vis_np = props.vis.to_numpy()
    diff_np = props.diff.to_numpy()
    den_np = props.den.to_numpy()
    T_np = state.temp.to_numpy()

    print("vis min/max:", float(vis_np.min()), float(vis_np.max()))
    print("diff min/max:", float(diff_np.min()), float(diff_np.max()))
    print("den min/max:", float(den_np.min()), float(den_np.max()))

    # 1) Liquid pocket: should have den ~ denl, vis ~ viscos (not 1e10)
    ic, jc, kc = sim.ni // 2, sim.nj // 2, sim.nk // 2
    print("\nLiquid pocket sample (center):")
    print("  T =", float(T_np[ic, jc, kc]))
    print("  den =", float(den_np[ic, jc, kc]), " expected ~", coeffs.denl)
    print("  vis =", float(vis_np[ic, jc, kc]), " expected ~", coeffs.viscos)

    # 2) Solid region (non-top): should have vis = 1e10, den = dens
    print("\nSolid sample (0,0,0):")
    print("  T =", float(T_np[0, 0, 0]))
    print("  den =", float(den_np[0, 0, 0]), " expected ~", coeffs.dens)
    print("  vis =", float(vis_np[0, 0, 0]), " expected 1e10")

    # 3) Mushy region: den should be mixture, vis should be viscos (not 1e10)
    print("\nMushy sample (i=1, j=0, k=0):")
    print("  T =", float(T_np[1, 0, 0]))
    print("  den =", float(den_np[1, 0, 0]), " expected ~", 0.5 * (coeffs.denl + coeffs.dens))
    print("  vis =", float(vis_np[1, 0, 0]), " expected ~", coeffs.viscos)

    # 4) Powder top layer: den should be pden, vis should be 1e10
    print("\nPowder top layer sample (0,0,nk-1):")
    print("  T =", float(T_np[0, 0, sim.nk - 1]))
    print("  den =", float(den_np[0, 0, sim.nk - 1]), " expected ~", coeffs.pden)
    print("  vis =", float(vis_np[0, 0, sim.nk - 1]), " expected 1e10")


if __name__ == "__main__":
    main()


[Taichi] Starting on arch=x64
vis min/max: 0.006 10000000000.0
diff min/max: 0.01 0.05
den min/max: 4000.0 7800.0

Liquid pocket sample (center):
  T = 1673.0
  den = 7800.0  expected ~ 7800.0
  vis = 0.006  expected ~ 0.006

Solid sample (0,0,0):
  T = 1513.0
  den = 7800.0  expected ~ 7800.0
  vis = 10000000000.0  expected 1e10

Mushy sample (i=1, j=0, k=0):
  T = 1593.0
  den = 7800.0  expected ~ 7800.0
  vis = 0.006  expected ~ 0.006

Powder top layer sample (0,0,nk-1):
  T = 1463.0
  den = 4000.0  expected ~ 4000.0
  vis = 10000000000.0  expected 1e10
